In [1]:
import pandas as pd 
import numpy as np 
import tensorflow as tf

from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Dropout, Bidirectional
from modules.utils import  change_X_data_2


In [2]:
X = pd.read_csv("/Users/elouan/Data/Data_ENSxCFM/X_train.csv")
y = pd.read_csv("/Users/elouan/Data/Data_ENSxCFM/y_train.csv")

In [3]:
X2 = change_X_data_2(X)


/Users/elouan/anaconda3/envs/challenge_ENS_env_CFM/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [4]:
X3 = np.array([np.array(row) for row in X2["column 1"]])

In [5]:
y1 = y.drop(columns=["obs_id"])

In [7]:
y2 = np.array(y1["eqt_code_cat"])

In [10]:
print(np.any(np.isnan(X3)))  # Check features for NaNs
print(np.any(np.isnan(y2)))

False
False


In [9]:
X3 = np.nan_to_num(X3)

In [11]:
dataset = tf.data.Dataset.from_tensor_slices((X3, y2))

In [12]:
# Calculate sizes for train and validation sets
dataset_size = len(X3)  # Total number of samples in your dataset
train_size = int(0.8 * dataset_size)

# Shuffle the dataset if needed
dataset = dataset.shuffle(buffer_size=dataset_size)

# Split the dataset
train_dataset = dataset.take(train_size)       # Take the first 80%
val_dataset = dataset.skip(train_size)         # Skip the first 80%, take the remaining 20%

# Batch both datasets if needed
batch_size = 32
train_dataset = train_dataset.batch(batch_size)
val_dataset = val_dataset.batch(batch_size)

In [25]:
tf.random.set_seed(1234) # for consistent results
model = Sequential(
    [   Bidirectional(GRU(128, return_sequences=True, input_shape = (100, 11) )),            
        Dropout(0.2),
        Bidirectional(GRU(64)),
        Dropout(0.2),
        Dense(64, activation = 'relu',   name = "L1"),
        Dense(32, activation = 'relu',   name = "L2"),
        Dense(24, activation = 'softmax',   name = "output"),

    ], name = "second_multiclass_model" 
)

/Users/elouan/anaconda3/envs/challenge_ENS_env_CFM/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [29]:
model.summary()

Model: "second_multiclass_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ bidirectional_8 (Bidirectional) │ (None, 100, 256)       │       108,288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 100, 256)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_9 (Bidirectional) │ (None, 128)            │       123,648 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ L1 (Dense)                      │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ L2 (Dense)                      │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output (Dense)                  │ (None, 24)             │           792 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 729,194 (2.78 MB)

 Trainable params: 243,064 (949.47 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 486,130 (1.85 MB)

In [35]:
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    metrics=['accuracy']
)




reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',  # Choose the metric to monitor
    factor=0.5,          # Factor by which to reduce the learning rate
    patience=3,          # Number of epochs with no improvement before reducing
    min_lr=1e-6          # Minimum learning rate
)



history = model.fit(
    train_dataset,
    validation_data = val_dataset,
    epochs=15,
    callbacks=[reduce_lr]
)

Epoch 1/15
4020/4020 ━━━━━━━━━━━━━━━━━━━━ 1056s 261ms/step - accuracy: 0.6969 - loss: 0.8681 - val_accuracy: 0.7571 - val_loss: 0.7022 - learning_rate: 0.0010
Epoch 2/15
4020/4020 ━━━━━━━━━━━━━━━━━━━━ 998s 248ms/step - accuracy: 0.6917 - loss: 0.8873 - val_accuracy: 0.7451 - val_loss: 0.7386 - learning_rate: 0.0010
Epoch 3/15
4020/4020 ━━━━━━━━━━━━━━━━━━━━ 963s 239ms/step - accuracy: 0.6901 - loss: 0.8867 - val_accuracy: 0.7566 - val_loss: 0.7016 - learning_rate: 0.0010
Epoch 4/15
4020/4020 ━━━━━━━━━━━━━━━━━━━━ 974s 242ms/step - accuracy: 0.6952 - loss: 0.8795 - val_accuracy: 0.7476 - val_loss: 0.7230 - learning_rate: 0.0010
Epoch 5/15
4020/4020 ━━━━━━━━━━━━━━━━━━━━ 973s 242ms/step - accuracy: 0.6938 - loss: 0.8828 - val_accuracy: 0.7440 - val_loss: 0.7492 - learning_rate: 0.0010
Epoch 6/15
4020/4020 ━━━━━━━━━━━━━━━━━━━━ 961s 239ms/step - accuracy: 0.6912 - loss: 0.8886 - val_accuracy: 0.7456 - val_loss: 0.7412 - learning_rate: 0.0010
Epoch 7/15
4020/4020 ━━━━━━━━━━━━━━━━━━━━ 978s 243m

In [37]:
model.save("../models/second_model_4.h5") 